# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [36]:
# Write your code below.
%load_ext dotenv
%dotenv 01_materials\labs\.env

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [43]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [52]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
print(PRICE_DATA)
parquet_files = glob(os.path.join(PRICE_DATA, "*/*.parquet"))
print(len(parquet_files))


../../05_src/data/prices/
558


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [48]:
# Write your code below.
import numpy as np
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
).assign(
    positive_return = lambda x: (x['returns'] > 0)*1
).assign(
    hi_lo_range = lambda x: x['High'] - x['Low']
))

C:\Users\parva\AppData\Local\Temp\ipykernel_34676\2489636934.py:4: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [61]:
# Write your code below.
# covert to pd
dd_pd = dd_feat.compute()
dd_pd['rolling_mean'] = dd_pd['returns'].rolling(10).mean()
dd_pd



,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year,Close_lag_1,returns,positive_return,hi_lo_range,rolling_mean
ticker,,,,,,,,,,,,,,
A,2013-12-02 00:00:00-05:00,34.995827,35.126724,34.760209,34.825657,2039962,0.0,0.0,2013,NaN,NaN,0,0.366515,NaN
A,2013-12-03 00:00:00-05:00,34.635858,34.799484,34.400240,34.642403,3462706,0.0,0.0,2013,34.825657,-0.005262,0,0.399244,NaN
A,2013-12-04 00:00:00-05:00,34.583499,35.231451,34.504962,35.067825,3377288,0.0,0.0,2013,34.642403,0.012280,1,0.726489,NaN
A,2013-12-05 00:00:00-05:00,34.917279,35.211801,34.766746,35.015453,2530939,0.0,0.0,2013,35.067825,-0.001493,0,0.445055,NaN
A,2013-12-06 00:00:00-05:00,35.185629,35.951385,35.185629,35.885937,4268513,0.0,0.0,2013,35.015453,0.024860,1,0.765756,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,2024-01-25 00:00:00-05:00,128.551783,129.027053,127.274475,128.937943,1871700,0.0,0.0,2024,128.165619,0.006026,1,1.752578,0.000828
YUM,2024-01-26 00:00:00-05:00,128.640891,129.403315,127.403198,127.819061,1157000,0.0,0.0,2024,128.937943,-0.008678,0,2.000117,0.000115
YUM,2024-01-29 00:00:00-05:00,127.828973,129.264694,127.026935,129.264694,1482100,0.0,0.0,2024,127.819061,0.011310,1,2.237759,0.001145


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
- It is not necessary to convert to pandas to calculate the moving average.

+ Would it have been better to do it in Dask? Why?
- For a smaller dataset using pandas with the builting in rolling mean fucntion might be advantageous and simpler, But larger dataset dask would be better. 

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.